In [37]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import numpy as np
from pyspark.sql.functions import round
from pyspark.sql import Window
from pyspark.sql.functions import col, when, log
from pyspark.sql.functions import regexp_replace

# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 Tutorial 1")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .getOrCreate()
)

In [2]:
final = spark.read.parquet('../data/curated/merchant_info.parquet')
final_1 = final

In [3]:
col_list_1 = final.columns

del_col_list = ['Store_type', 'Revenue_levels', 'merchant_abn','postcode']

for i in del_col_list:
    col_list_1.remove(i)

In [4]:
max_num_list = []
min_num_list = []

for i in col_list_1:
    max_num = float(final.agg({i: "max"}).collect()[0][0])
    min_num = float(final.agg({i: "min"}).collect()[0][0])
    max_num_list.append(max_num)
    min_num_list.append(min_num)

min_num_list

[0.1, 0.0, 165.96, 0.06, 5.19, 3.7259495856741, 0.0, 0.06, 0.0, 0.0, 0.0]

In [5]:
for i in range(0,len(min_num_list)):
    final_1 = final_1.withColumn(col_list_1[i], (col(col_list_1[i]) - min_num_list[i])*1000/(max_num_list[i]-min_num_list[i]))

final_1 = final_1.withColumn('avg_prob_fraud_cus', (final_1.avg_prob_fraud_cus *(-1)))\
    .withColumn('num_of_fraud', (final_1.num_of_fraud*(-1)))
    
final_1 = final_1.select('merchant_abn','Store_type', 'Revenue_levels', 'Take_rate', 'count_of_bigorder', 'Avg_amount_monthly', 'Avg_count_monthly', 'Order_avg_value',	'avg_prob_fraud_cus', 'num_of_fraud', 'count_cus_per_mon', 'fix_cus_num', 'avg_income_percentage', 'avg_age_percentage')

In [6]:
final_1

merchant_abn,Store_type,Revenue_levels,Take_rate,count_of_bigorder,Avg_amount_monthly,Avg_count_monthly,Order_avg_value,avg_prob_fraud_cus,num_of_fraud,count_cus_per_mon,fix_cus_num,avg_income_percentage,avg_age_percentage
10023283211,"furniture, home f...",e,11.594202898550723,0.0,55.37615475655712,11.03555286822219,3.353289719103669,-103.0741890985961,-97.5609756097561,112.16427338032696,0.0,234.61213236592934,488.65282472235634
10142254217,"cable, satellite,...",b,597.1014492753623,0.0,6.950906119841834,9.900798745420259,0.4025335709720858,-104.52711111927987,-146.34146341463415,101.28796429879576,0.0,488.3827843314614,737.5104079933388
10165489824,"jewelry, watch, c...",b,623.1884057971015,13.88888888888889,5.745141451931026,0.015185274149417576,216.51409397655166,-160.30259518888823,-0.0,0.16445278336335845,0.0,445.9369480915662,583.7680097854485
10187291046,"watch, clock, and...",b,462.3188405797101,0.0,2.7394004319538876,1.1347541228019316,1.7402636710560435,-89.55070406580317,-12.195121951219512,12.244257234053688,0.0,302.76340770958546,540.9193246435595
10192359162,music shops - mus...,a,902.8985507246376,0.0,12.853635659839755,1.292128782168623,6.858300664431012,-107.570852401475,-12.195121951219512,13.948586079819401,0.0,549.4475276974022,805.4845980465816
10206519221,"gift, card, novel...",a,904.3478260869565,0.0,22.36333607566935,31.64335036844997,0.3905809457803859,-102.70985794242861,-365.8536585365854,289.3247718217631,0.0,441.1437452621542,566.1795407098122
10255988167,"computers, comput...",b,611.5942028985509,0.0,23.28162787678455,2.734039586629228,5.800878903520145,-99.17308774178454,-12.195121951219512,29.06329189621534,0.0,394.38218025600224,515.5781905332535
10264435225,"watch, clock, and...",c,331.8840579710145,0.0,43.532824766673016,16.764542660957,1.6899469746845328,-98.60814577621464,-97.5609756097561,166.03003505834334,0.0,673.0942474685537,805.8275427240694
10279061213,computer programm...,a,813.0434782608695,0.0,12.21955065115608,1.8132597813872713,4.6229669693058515,-100.80252826581189,-36.58536585365854,19.517555334624042,0.0,414.40206552058595,550.4564520108561
10323485998,"furniture, home f...",a,943.4782608695652,0.0,93.49281729980491,34.32355125582217,1.770530802589864,-102.66711493699285,-304.8780487804878,311.4212458045852,0.04220834036805673,290.22187281980075,554.8612846788303


In [7]:
reve_list = ['a', 'b', 'c', 'd', 'e']
reve_num = [0, 250, 500, 750, 1000]
for i in range(len(reve_list)):
    final_1 = final_1.withColumn('Revenue_levels', when(col('Revenue_levels') == reve_list[i], reve_num[i]).otherwise(col('Revenue_levels')))

In [8]:
from operator import add
from functools import reduce
final_df1 = final_1.withColumn('score',reduce(add, [F.col(x) for x in final_1.columns[2:14]]))

In [9]:
final_df1

merchant_abn,Store_type,Revenue_levels,Take_rate,count_of_bigorder,Avg_amount_monthly,Avg_count_monthly,Order_avg_value,avg_prob_fraud_cus,num_of_fraud,count_cus_per_mon,fix_cus_num,avg_income_percentage,avg_age_percentage,score
10023283211,"furniture, home f...",1000,11.594202898550723,0.0,55.37615475655712,11.03555286822219,3.353289719103669,-103.0741890985961,-97.5609756097561,112.16427338032696,0.0,234.61213236592934,488.65282472235634,1716.153266002694
10142254217,"cable, satellite,...",250,597.1014492753623,0.0,6.950906119841834,9.900798745420259,0.4025335709720858,-104.52711111927987,-146.34146341463415,101.28796429879576,0.0,488.3827843314614,737.5104079933388,1940.6682698012783
10165489824,"jewelry, watch, c...",250,623.1884057971015,13.88888888888889,5.745141451931026,0.015185274149417576,216.51409397655166,-160.30259518888823,-0.0,0.16445278336335845,0.0,445.9369480915662,583.7680097854485,1978.9185308601122
10187291046,"watch, clock, and...",250,462.3188405797101,0.0,2.7394004319538876,1.1347541228019316,1.7402636710560435,-89.55070406580317,-12.195121951219512,12.244257234053688,0.0,302.76340770958546,540.9193246435595,1472.1144223756978
10192359162,music shops - mus...,0,902.8985507246376,0.0,12.853635659839755,1.292128782168623,6.858300664431012,-107.570852401475,-12.195121951219512,13.948586079819401,0.0,549.4475276974022,805.4845980465816,2173.0173533021857
10206519221,"gift, card, novel...",0,904.3478260869565,0.0,22.36333607566935,31.64335036844997,0.3905809457803859,-102.70985794242861,-365.8536585365854,289.3247718217631,0.0,441.1437452621542,566.1795407098122,1786.8296347915716
10255988167,"computers, comput...",250,611.5942028985509,0.0,23.28162787678455,2.734039586629228,5.800878903520145,-99.17308774178454,-12.195121951219512,29.06329189621534,0.0,394.38218025600224,515.5781905332535,1721.0662022579518
10264435225,"watch, clock, and...",500,331.8840579710145,0.0,43.532824766673016,16.764542660957,1.6899469746845328,-98.60814577621464,-97.5609756097561,166.03003505834334,0.0,673.0942474685537,805.8275427240694,2342.6540762383247
10279061213,computer programm...,0,813.0434782608695,0.0,12.21955065115608,1.8132597813872713,4.6229669693058515,-100.80252826581189,-36.58536585365854,19.517555334624042,0.0,414.40206552058595,550.4564520108561,1678.6874344093142
10323485998,"furniture, home f...",0,943.4782608695652,0.0,93.49281729980491,34.32355125582217,1.770530802589864,-102.66711493699285,-304.8780487804878,311.4212458045852,0.04220834036805673,290.22187281980075,554.8612846788303,1822.066608153886


In [10]:
final_df1 = final_df1.sort(final_df1.score.desc())

In [11]:
final_df1

merchant_abn,Store_type,Revenue_levels,Take_rate,count_of_bigorder,Avg_amount_monthly,Avg_count_monthly,Order_avg_value,avg_prob_fraud_cus,num_of_fraud,count_cus_per_mon,fix_cus_num,avg_income_percentage,avg_age_percentage,score
64203420245,tent and awning s...,500,400.0,0.0,815.055972932642,1000.0,0.45805544283030447,-103.0026184437547,-1000.0,1000.0,1000.0,520.2931741035037,671.5831110113019,4804.387695046524
49891706470,tent and awning s...,0,826.0869565217391,0.0,772.791473710687,948.2354711438376,0.45805544283030447,-103.00308583547402,-1000.0,999.880397975736,991.0096235016039,414.5546579090338,518.2701782093046,4368.283728579298
24852446429,florists supplies...,500,411.5942028985507,0.0,648.6912042945456,966.1741115924382,0.35973546141470886,-103.00004568869598,-1000.0,999.9177736083184,994.4284990714165,403.7962238931629,534.3330702446725,4356.2947753758235
86578477987,"watch, clock, and...",0,917.391304347826,0.0,708.3793742893614,908.8766210280155,0.43357183961504836,-103.00085255309646,-1000.0,999.9626243674174,985.3959142326523,327.7999166601939,567.8988503398783,4313.1373245518635
43186523025,florists supplies...,250,633.3333333333333,0.0,719.6055911927039,675.3381484457182,0.6296334496143834,-103.03426592884695,-1000.0,999.2524873483484,868.0567280094547,520.2931741035037,671.5831110113019,4235.057940965131
46804135891,"opticians, optica...",500,410.1449275362319,0.0,438.3968606182158,769.4102315616259,0.2901403373146499,-103.00587786087326,-1000.0,999.6262436741742,933.6062806010468,520.2931741035037,671.5831110113019,4140.345091582542
45629217853,"gift, card, novel...",0,997.1014492753624,0.0,522.6523058932152,749.0398765299165,0.3778571834795441,-102.99720668418352,-1000.0,999.8355472166367,924.826945804491,353.96825075035065,597.2427375677006,4042.04776353697
89726005175,tent and awning s...,0,856.5217391304348,0.0,708.5737834776995,725.7366928681669,0.5685208337148856,-102.99941866865255,-1000.0,999.6262436741742,905.8754009792335,403.7962238931629,534.3330702446725,4032.032256432606
19492220327,"jewelry, watch, c...",250,700.0,1000.0,597.3107999086589,2.7181640727457452,150.146371462529,-135.09921587755102,-0.0,28.898839112851984,0.0,549.4475276974022,805.4845980465816,3948.907084423218
80324045558,"gift, card, novel...",500,198.55072463768113,0.0,448.11090453190764,645.6764763537673,0.3753509878748329,-103.02503940015349,-1000.0,998.9235817816218,840.9589734931623,520.2931741035037,671.5831110113019,3721.4472575006666


In [12]:
col_list_2 = final_df1.columns

del_col_list = ['Store_type', 'merchant_abn','score']

for i in del_col_list:
    col_list_2.remove(i)

In [13]:
final_df2 = final_df1.withColumn('avg_prob_fraud_cus', (final_df1.avg_prob_fraud_cus *(-1)))\
    .withColumn('num_of_fraud', (final_df1.num_of_fraud*(-1)))

In [14]:
sum_num_list = []

for i in col_list_2:
    sum_num = float(final_df2.agg({i: "sum"}).collect()[0][0])
    sum_num_list.append(sum_num)

In [15]:
for i in range(0,len(sum_num_list)):
    final_df2 = final_df2.withColumn(col_list_2[i], (col(col_list_2[i])/sum_num_list[i]))

In [16]:
col_list_2 = final_df2.columns

del_col_list = ['Store_type', 'merchant_abn', 'score']

for i in del_col_list:
    col_list_2.remove(i)

k = 1/np.log(final_df2.count())

for i in col_list_2:
    final_df2 = final_df2.withColumn(i, when((col(i) != 0), (col(i) * log(col(i)) *(-k))).otherwise(0))


In [17]:
weight_num = []

for i in col_list_2:
    a = 1 - float(final_df2.agg({i: "sum"}).collect()[0][0])
    weight_num.append(a)

entropy = []
total = sum(weight_num)
for i in weight_num:
    a = i/total
    entropy.append(a)

In [18]:
entropy

[0.03606848646131884,
 0.005929931728430069,
 0.2509361071617077,
 0.07023964853643903,
 0.10858296820604747,
 0.07943489081239004,
 0.0013956174122030437,
 0.07272573970663626,
 0.06849316688779244,
 0.30144953170794236,
 0.0038387533131199556,
 0.0009051580659728188]

In [30]:
final_df2 = final_1.withColumn('score', final_1.Revenue_levels * entropy[0]+final_1.Take_rate * entropy[1] + final_1.count_of_bigorder * entropy[2] +\
                            final_1.Avg_amount_monthly *entropy[3]+ final_1.Avg_count_monthly *entropy[4] +\
                            final_1.Order_avg_value *entropy[5] + final_1.avg_prob_fraud_cus * entropy[6]+ final_1.num_of_fraud * entropy[7]+\
                            final_1.count_cus_per_mon * entropy[8] + final_1.fix_cus_num  * entropy[9]+\
                            final_1.avg_income_percentage * entropy[10] + final_1.avg_age_percentage * entropy[11])

In [31]:
final_df2 = final_df2.sort(final_df2.score.desc())

In [32]:
final_df2

merchant_abn,Store_type,Revenue_levels,Take_rate,count_of_bigorder,Avg_amount_monthly,Avg_count_monthly,Order_avg_value,avg_prob_fraud_cus,num_of_fraud,count_cus_per_mon,fix_cus_num,avg_income_percentage,avg_age_percentage,score
64203420245,tent and awning s...,500,400.0,0.0,815.055972932642,1000.0,0.45805544283030447,-103.0026184437547,-1000.0,1000.0,1000.0,520.2931741035037,671.5831110113019,485.9531874455627
24852446429,florists supplies...,500,411.5942028985507,0.0,648.6912042945456,966.1741115924382,0.35973546141470886,-103.00004568869598,-1000.0,999.9177736083184,994.4284990714165,403.7962238931629,534.3330702446725,468.3992212604614
49891706470,tent and awning s...,0,826.0869565217391,0.0,772.791473710687,948.2354711438376,0.45805544283030447,-103.00308583547402,-1000.0,999.880397975736,991.0096235016039,414.5546579090338,518.2701782093046,458.5932071394061
86578477987,"watch, clock, and...",0,917.391304347826,0.0,708.3793742893614,908.8766210280155,0.43357183961504836,-103.00085255309646,-1000.0,999.9626243674174,985.3959142326523,327.7999166601939,567.8988503398783,448.35998361556506
46804135891,"opticians, optica...",500,410.1449275362319,0.0,438.3968606182158,769.4102315616259,0.2901403373146499,-103.00587786087326,-1000.0,999.6262436741742,933.6062806010468,520.2931741035037,671.5831110113019,414.4655227687114
89726005175,tent and awning s...,0,856.5217391304348,0.0,708.5737834776995,725.7366928681669,0.5685208337148856,-102.99941866865255,-1000.0,999.6262436741742,905.8754009792335,403.7962238931629,534.3330702446725,404.40441862487796
45629217853,"gift, card, novel...",0,997.1014492753624,0.0,522.6523058932152,749.0398765299165,0.3778571834795441,-102.99720668418352,-1000.0,999.8355472166367,924.826945804491,353.96825075035065,597.2427375677006,400.2871101300165
43186523025,florists supplies...,250,633.3333333333333,0.0,719.6055911927039,675.3381484457182,0.6296334496143834,-103.03426592884695,-1000.0,999.2524873483484,868.0567280094547,520.2931741035037,671.5831110113019,396.550715845839
80324045558,"gift, card, novel...",500,198.55072463768113,0.0,448.11090453190764,645.6764763537673,0.3753509878748329,-103.02503940015349,-1000.0,998.9235817816218,840.9589734931623,520.2931741035037,671.5831110113019,372.4878432906064
68216911708,"computers, comput...",500,427.5362318840579,0.0,487.93955382857985,599.5229062958148,0.4574770899984481,-103.01124803083026,-1000.0,998.549825455796,790.0135066689178,327.7999166601939,567.8988503398783,355.42244169305246


22/09/29 13:50:14 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 196376 ms exceeds timeout 120000 ms
22/09/29 13:50:14 WARN SparkContext: Killing executors is not supported by current scheduler.


In [22]:
#从这开始接着改

In [23]:
col_list_3 = final.columns

del_col_list = ['Store_type', 'Revenue_levels', 'merchant_abn','postcode']

for i in del_col_list:
    col_list_3.remove(i)

mean_num_list = []
sd_num_list = []

for i in col_list_3:
    mean_num = float(final.agg({i: "mean"}).collect()[0][0])
    sd_num = float(final.agg({i: "stddev_pop"}).collect()[0][0])
    mean_num_list.append(mean_num)
    sd_num_list.append(sd_num)


In [24]:
final_3 = final

for i in range(0,len(mean_num_list)):
    final_3 = final_3.withColumn(col_list_3[i], (col(col_list_3[i]) - mean_num_list[i])/(sd_num_list[i]))


final_3 = final_3.withColumn('avg_prob_fraud_cus', (final_3.avg_prob_fraud_cus *(-1)))\
    .withColumn('num_of_fraud', (final_3.num_of_fraud*(-1)))
    
final_3 = final_3.select('merchant_abn','Store_type', 'Revenue_levels', 'Take_rate', 'count_of_bigorder', 'Avg_amount_monthly', 'Avg_count_monthly', 'Order_avg_value',	'avg_prob_fraud_cus', 'num_of_fraud', 'count_cus_per_mon', 'fix_cus_num', 'avg_income_percentage', 'avg_age_percentage')

In [25]:
final_df3 = final_3.withColumn('score',reduce(add, [F.col(x) for x in final_1.columns[3:14]]))
final_df3 = final_df3.sort(final_df3.score.desc())

In [26]:
final_df_weight = final_df1.withColumn('weight_score', 0.05*11*final_df1.Take_rate + 0.025*11*final_df1.count_of_bigorder +\
                            0.15*11*final_df1.Avg_amount_monthly + 0.15*11*final_df1.Avg_count_monthly +\
                            0.025*11*final_df1.Order_avg_value + 0.25*11*final_df1.avg_prob_fraud_cus + 0.25*11*final_df1.num_of_fraud +\
                            0.1*11*final_df1.count_cus_per_mon + 0.05*11*final_df1.fix_cus_num +\
                            0.02*11*final_df1.avg_income_percentage + 0.02*11*final_df1.avg_age_percentage)

In [27]:
final_df_weight = final_df_weight.sort(final_df_weight.weight_score.desc())

In [28]:
final_df2

merchant_abn,Store_type,Revenue_levels,Take_rate,count_of_bigorder,Avg_amount_monthly,Avg_count_monthly,Order_avg_value,avg_prob_fraud_cus,num_of_fraud,count_cus_per_mon,fix_cus_num,avg_income_percentage,avg_age_percentage,score
64203420245,tent and awning s...,500,400.0,0.0,815.055972932642,1000.0,0.45805544283030447,-103.0026184437547,-1000.0,1000.0,1000.0,520.2931741035037,671.5831110113019,485.9531874455627
24852446429,florists supplies...,500,411.5942028985507,0.0,648.6912042945456,966.1741115924382,0.35973546141470886,-103.00004568869598,-1000.0,999.9177736083184,994.4284990714165,403.7962238931629,534.3330702446725,468.3992212604614
49891706470,tent and awning s...,0,826.0869565217391,0.0,772.791473710687,948.2354711438376,0.45805544283030447,-103.00308583547402,-1000.0,999.880397975736,991.0096235016039,414.5546579090338,518.2701782093046,458.5932071394061
86578477987,"watch, clock, and...",0,917.391304347826,0.0,708.3793742893614,908.8766210280155,0.43357183961504836,-103.00085255309646,-1000.0,999.9626243674174,985.3959142326523,327.7999166601939,567.8988503398783,448.35998361556506
46804135891,"opticians, optica...",500,410.1449275362319,0.0,438.3968606182158,769.4102315616259,0.2901403373146499,-103.00587786087326,-1000.0,999.6262436741742,933.6062806010468,520.2931741035037,671.5831110113019,414.4655227687114
89726005175,tent and awning s...,0,856.5217391304348,0.0,708.5737834776995,725.7366928681669,0.5685208337148856,-102.99941866865255,-1000.0,999.6262436741742,905.8754009792335,403.7962238931629,534.3330702446725,404.40441862487796
45629217853,"gift, card, novel...",0,997.1014492753624,0.0,522.6523058932152,749.0398765299165,0.3778571834795441,-102.99720668418352,-1000.0,999.8355472166367,924.826945804491,353.96825075035065,597.2427375677006,400.2871101300165
43186523025,florists supplies...,250,633.3333333333333,0.0,719.6055911927039,675.3381484457182,0.6296334496143834,-103.03426592884695,-1000.0,999.2524873483484,868.0567280094547,520.2931741035037,671.5831110113019,396.550715845839
80324045558,"gift, card, novel...",500,198.55072463768113,0.0,448.11090453190764,645.6764763537673,0.3753509878748329,-103.02503940015349,-1000.0,998.9235817816218,840.9589734931623,520.2931741035037,671.5831110113019,372.4878432906064
68216911708,"computers, comput...",500,427.5362318840579,0.0,487.93955382857985,599.5229062958148,0.4574770899984481,-103.01124803083026,-1000.0,998.549825455796,790.0135066689178,327.7999166601939,567.8988503398783,355.42244169305246


### rank inside each column and add up the rank

In [29]:
final_rank_coldf = final_1.drop("Take_rate", "Store_type", "Revenue_levels")
for col in final_rank_coldf.columns:
    if col == "merchant_abn":
        continue
    else: #columns that have number bigger = higher rank
        final_rank_coldf = final_rank_coldf.withColumn("rank_"+col, F.dense_rank().over(Window.orderBy(F.desc(col)))).drop(col)

numeric_col = final_rank_coldf.columns
numeric_col.remove("merchant_abn")
expression = '+'.join(numeric_col)

final_rank_coldf = final_rank_coldf.withColumn("sum_of_ranks", F.expr(expression))\
    .withColumn("rank", F.dense_rank().over(Window.orderBy("sum_of_ranks")))
final_rank_coldf.orderBy("rank").show(20) 

22/09/29 13:37:24 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/09/29 13:37:24 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/09/29 13:37:24 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/09/29 13:37:24 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/09/29 13:37:24 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/09/29 13:37:24 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
22/09/29 1

+------------+----------------------+-----------------------+----------------------+--------------------+-----------------------+-----------------+----------------------+----------------+--------------------------+-----------------------+------------+----+
|merchant_abn|rank_count_of_bigorder|rank_Avg_amount_monthly|rank_Avg_count_monthly|rank_Order_avg_value|rank_avg_prob_fraud_cus|rank_num_of_fraud|rank_count_cus_per_mon|rank_fix_cus_num|rank_avg_income_percentage|rank_avg_age_percentage|sum_of_ranks|rank|
+------------+----------------------+-----------------------+----------------------+--------------------+-----------------------+-----------------+----------------------+----------------+--------------------------+-----------------------+------------+----+
| 31334588839|                     5|                     45|                  1099|                 180|                    474|               76|                  1099|             131|                        98|               

In [54]:
final.select('Store_type').distinct().toPandas().values.tolist()

[['opticians, optical goods, and eyeglasses'],
 ['watch, clock, and jewelry repair shops'],
 ['computer programming , data processing, and integrated systems design services'],
 ['digital goods: books, movies, music'],
 ['books, periodicals, and newspapers'],
 ['florists supplies, nursery stock, and flowers'],
 ['art dealers and galleries'],
 ['antique shops - sales, repairs, and restoration services'],
 ['gift, card, novelty, and souvenir shops'],
 ['equipment, tool, furniture, and appliance rent al and leasing'],
 ['cable, satellite, and other pay television and radio services'],
 ['tent and awning shops'],
 ['artist supply and craft shops'],
 ['stationery, office supplies and printing and writing paper'],
 ['furniture, home furnishings and equipment shops, and manufacturers, except appliances'],
 ['telecom'],
 ['jewelry, watch, clock, and silverware shops'],
 ['shoe shops'],
 ['hobby, toy and game shops'],
 ['computers, computer peripheral equipment, and software'],
 ['music shops -

In [55]:
art = final.filter((final.Store_type == 'antique shops - sales, repairs, and restoration services') 
                    | (final.Store_type == 'art dealers and galleries')
                    | (final.Store_type == 'watch, clock, and jewelry repair shops')
                    | (final.Store_type == 'artist supply and craft shops')
                    | (final.Store_type == 'jewelry, watch, clock, and silverware shops')
                    | (final.Store_type == 'music shops - musical instruments, pianos, and sheet music'))


In [57]:
sports = final.filter((final.Store_type == 'bicycle shops - sales and service') 
                    | (final.Store_type == 'motor vehicle supplies and new parts')
                    | (final.Store_type == 'tent and awning shops'))

In [59]:
technology = final.filter((final.Store_type == 'cable, satellite, and other pay television and radio services') 
                    | (final.Store_type == 'computer programming , data processing, and integrated systems design services')
                    | (final.Store_type == 'computers, computer peripheral equipment, and software')
                    | (final.Store_type == 'digital goods: books, movies, music')
                    | (final.Store_type == 'equipment, tool, furniture, and appliance rent al and leasing')
                    | (final.Store_type == 'telecom'))

In [60]:
home = final.filter((final.Store_type == 'books, periodicals, and newspapers') 
                    | (final.Store_type == 'florists supplies, nursery stock, and flowers')
                    | (final.Store_type == 'furniture, home furnishings and equipment shops, and manufacturers, except appliances')
                    | (final.Store_type == 'gift, card, novelty, and souvenir shops')
                    | (final.Store_type == 'hobby, toy and game shops')
                    | (final.Store_type == 'lawn and garden supply outlets, including nurseries')
                    | (final.Store_type == 'shoe shops')
                    | (final.Store_type == 'stationery, office supplies and printing and writing paper'))

In [61]:
health = final.filter((final.Store_type == 'health and beauty spas') 
                    | (final.Store_type == 'opticians, optical goods, and eyeglasses'))